In [34]:
import numpy as np
import pandas as pd
import sqlite3
import pickle

with open("../adagrid/4d/2090.pkl", "rb") as f:
    S = pickle.load(f)
# all_data = np.concatenate((S.db.data, S.sim_sizes[:, None], S.todo[:, None], S.g.grid_pt_idx[:, None], S.g.null_truth), axis=1)
# np.save('2090.npy', all_data)

In [73]:
df = pd.DataFrame(S.db.data)

In [74]:
rename_dict = dict()
for k, v in S.db.slices.items():
    if isinstance(v, slice):
        for i in range(v.start, v.stop, 1 if v.step is None else v.step):
            rename_dict[i] = k + "_" + str(i)
    else:
        rename_dict[v] = k
df.rename(columns=rename_dict, inplace=True)

In [75]:
df["sim_sizes"] = S.sim_sizes
df["todo"] = S.todo
df["grid_pt_idx"] = S.g.grid_pt_idx
for d in range(S.g.d):
    df[f"theta_{d}"] = S.g.thetas[S.g.grid_pt_idx, d]
    df[f"radii_{d}"] = S.g.radii[S.g.grid_pt_idx, d]

for i in range(S.g.null_truth.shape[1]):
    df[f"null_truth_{i}"] = S.g.null_truth[:, i]

In [76]:
df.head()

,alpha0,orig_lam,B_lam_2,B_lam_3,B_lam_4,B_lam_5,B_lam_6,B_lam_7,B_lam_8,B_lam_9,...,radii_0,theta_1,radii_1,theta_2,radii_2,theta_3,radii_3,null_truth_0,null_truth_1,null_truth_2
0,0.004738,0.091053,0.092897,0.080592,0.097186,0.102855,0.102855,0.091053,0.100334,0.091053,...,0.031250,0.906250,0.031250,0.406250,0.031250,-0.968750,0.031250,False,True,True
1,0.004738,0.071704,0.076335,0.071704,0.066958,0.065801,0.073842,0.071327,0.070249,0.066504,...,0.031250,0.906250,0.031250,0.406250,0.031250,-0.968750,0.031250,False,True,True
2,0.011102,0.089806,0.093288,0.090544,0.103630,0.089806,0.099318,0.074713,0.104047,0.090320,...,0.015625,-0.046875,0.015625,-0.640625,0.015625,-0.859375,0.015625,True,True,True
3,0.011101,0.092243,0.093288,0.092587,0.099708,0.092243,0.097845,0.069517,0.100998,0.092354,...,0.015625,-0.046875,0.015625,-0.671875,0.015625,-0.828125,0.015625,True,True,True
4,0.011111,0.089011,0.090544,0.089232,0.096866,0.089011,0.096798,0.069001,0.099708,0.090320,...,0.015625,-0.046875,0.015625,-0.703125,0.015625,-0.828125,0.015625,True,True,True


In [77]:
%%time
RR = df["grid_pt_idx"].sample(frac=0.1)

CPU times: user 392 ms, sys: 28.4 ms, total: 421 ms
Wall time: 421 ms


In [62]:
%%time
RR.shape, df["grid_pt_idx"].isin(RR).sum()

CPU times: user 319 ms, sys: 18.3 ms, total: 337 ms
Wall time: 337 ms


((1739069,), 1741991)

In [63]:
df.columns

Index(['alpha0', 'orig_lam', 'B_lam_2', 'B_lam_3', 'B_lam_4', 'B_lam_5',
       'B_lam_6', 'B_lam_7', 'B_lam_8', 'B_lam_9', 'B_lam_10', 'B_lam_11',
       'B_lam_12', 'B_lam_13', 'B_lam_14', 'B_lam_15', 'B_lam_16', 'B_lam_17',
       'B_lam_18', 'B_lam_19', 'B_lam_20', 'B_lam_21', 'B_lam_22', 'B_lam_23',
       'B_lam_24', 'B_lam_25', 'B_lam_26', 'B_lam_27', 'B_lam_28', 'B_lam_29',
       'B_lam_30', 'B_lam_31', 'B_lam_32', 'B_lam_33', 'B_lam_34', 'B_lam_35',
       'B_lam_36', 'B_lam_37', 'B_lam_38', 'B_lam_39', 'B_lam_40', 'B_lam_41',
       'B_lam_42', 'B_lam_43', 'B_lam_44', 'B_lam_45', 'B_lam_46', 'B_lam_47',
       'B_lam_48', 'B_lam_49', 'B_lam_50', 'B_lam_51', 'twb_min_lam',
       'twb_mean_lam', 'twb_max_lam', 'sim_sizes', 'todo', 'grid_pt_idx',
       'theta_0', 'theta_1', 'theta_2', 'theta_3', 'radii_0', 'radii_1',
       'radii_2', 'radii_3', 'null_truth_0', 'null_truth_1', 'null_truth_2'],
      dtype='object')

In [6]:
all_data = np.load("2090.npy")

## SQLite

In [7]:
con = sqlite3.connect("tutorial.db")

In [8]:
con.execute("DROP TABLE tiles")
con.execute("CREATE TABLE tiles(a REAL, b REAL, c REAL)")
con.execute("CREATE INDEX 'tiles_ordering' ON tiles(b)")
con.commit()

In [9]:
rows = all_data[: int(1e6), :3]

In [10]:
%%time
con.executemany("INSERT INTO tiles VALUES (?, ?, ?)", rows)
con.execute("select count(*) from tiles").fetchall()
con.commit()

CPU times: user 1.72 s, sys: 165 ms, total: 1.88 s
Wall time: 1.89 s


In [11]:
%%time
np.array(con.execute("select * from tiles order by b limit 1000000").fetchall())

CPU times: user 745 ms, sys: 175 ms, total: 920 ms
Wall time: 921 ms


array([[0.02384287, 0.06661493, 0.06657071],
       [0.02384294, 0.06661493, 0.06657071],
       [0.02384302, 0.06661493, 0.06657071],
       ...,
       [0.01119357, 0.17071873, 0.19691087],
       [0.01119357, 0.17071873, 0.19691087],
       [0.01119357, 0.17071873, 0.19691087]])

In [32]:
all_data[]

array([[4.73778974e-03, 9.10531282e-02, 9.28967297e-02, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [4.73778974e-03, 7.17036650e-02, 7.63351843e-02, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.11023095e-02, 8.98064300e-02, 9.32878554e-02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.38109808e-02, 1.25575259e-01, 1.45150363e-01, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.38109808e-02, 1.25208780e-01, 1.39460012e-01, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [2.38109808e-02, 1.00000000e+02, 1.00000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

## duckdb

In [25]:
import duckdb
import pyarrow as pa
import pandas as pd

con = duckdb.connect(database=":memory:")
# con.execute("DROP TABLE tiles")
con.execute("CREATE TABLE tiles(a REAL, b REAL, c REAL)")
con.execute("CREATE INDEX tiles_ordering ON tiles(b)")
con.commit()

In [30]:
%%time
tbl = pa.Table.from_pandas(pd.DataFrame(rows))
con.execute("insert into tiles select * from tbl").fetchall()

CPU times: user 242 ms, sys: 8.41 ms, total: 250 ms
Wall time: 215 ms


[(1000000,)]

In [29]:
%%time
con.execute("select * from tiles order by b limit 1000000").fetchnumpy()

CPU times: user 290 ms, sys: 39.3 ms, total: 329 ms
Wall time: 169 ms


{'a': array([0.02384287, 0.02384294, 0.02384302, ..., 0.02382353, 0.02382359,
        0.02382366], dtype=float32),
 'b': array([0.06661493, 0.06661493, 0.06661493, ..., 0.0729733 , 0.0729733 ,
        0.0729733 ], dtype=float32),
 'c': array([0.06657071, 0.06657071, 0.06657071, ..., 0.07032984, 0.07032984,
        0.07032984], dtype=float32)}

## redis??

In [1]:
import redis

In [2]:
r = redis.Redis(host="localhost", port=6379, db=0)

In [3]:
r.set("foo", "bar")

True

In [4]:
r.get("foo")

b'bar'